In [1]:
from __future__ import division
from __future__ import print_function

from collections import defaultdict
import math
import numpy as np
import pandas as pd

In [2]:
# A script for transforming scores from the originals
# ensuring students all had the same means/standard devs for their marker
# to counter harsh/easy marking and high/low variance

In [3]:
# Get the results file into a pandas data frame,
# Format should be:
# student_name, grader_name, q1_mark, q2_mark,....qn_mark
results_df = pd.read_csv("results.csv")
target_marker = "mary"

In [4]:
marker_dict = defaultdict(dict) # a collection of the markers' grades for each question
grade_dict = defaultdict(float) # a collection of the students' grades for each question

In [5]:
num_students = results_df.shape[0]
questions = list(results_df.columns)[2:]

In [6]:
for i in range(0, num_students):
    # iterate over the questions
    for q in questions:
        student = results_df['student_name'][i]
        grader = results_df['marker_name'][i]
        mark = results_df[q][i]
        # creat the dicts for graders and students if not there already
        if not marker_dict[grader]:
            marker_dict[grader] = defaultdict(list)
        if not grade_dict[student]:
            grade_dict[student] = defaultdict(float)
            
        marker_dict[grader][q].append(float(mark))
        grade_dict[student][q] = float(mark)

In [7]:
# decide on the target mean and st deviation for each question
# this might be something you decide yourself or pick the best marker
target_means_std = {}
for q in questions:
    target_means_std[q] = (np.mean(marker_dict[target_marker][q]), np.std(marker_dict[target_marker][q]))

In [8]:
print("target means and stds")
print(target_means_std)

target means and stds
{'q1': (14.7, 1.3453624047073711), 'q3': (17.1, 2.9137604568666933), 'q2': (15.1, 1.8138357147217055)}


In [9]:
# get all marker means and stds for each q
grader_means_std = {}
for grader in marker_dict.keys():
    this_grader_means_std = {}
    for q in questions:
        this_grader_means_std[q] = (np.mean(marker_dict[grader][q]), np.std(marker_dict[grader][q]))
    grader_means_std[grader]  = this_grader_means_std

In [10]:
print("grader means")
print(grader_means_std)

grader means
{'john': (10.3, 2.41039415863879), 'mary': (17.1, 2.9137604568666933), 'bill': (9.5, 8.523496934944014)}


In [12]:
# normalize the scores according to target mean and std
# replace all the markers with the target
normalized_grade_dict = defaultdict(float) # a collection of the students' grades for each question
for i in range(0, num_students):
    # iterate over the questions
    for q in questions:
        student = results_df['student_name'][i]
        grader = results_df['marker_name'][i]
        mark = results_df[q][i]
        
        if not normalized_grade_dict[student]:
            normalized_grade_dict[student] = defaultdict(float)
            
        # retrieve grader's average and std for this question
        grader_mean, grader_std = grader_means_std[grader][q]
        
        # get z-score for this based on marker average
        z = (mark - grader_mean) / grader_std
        if math.isnan(z):
            z = 0
            
        # now get the transformed t score based on target mean
        target_mean, target_std = target_means_std[q]
        if math.isnan(target_std):
            target_std = 0
        t = target_mean + (target_std * z)
        #print(z,t)
        new_mark = t
        # if you want to impose a max mark on a q:
        # new_mark = min([marks_max[q], int(new_mark)])
        normalized_grade_dict[student][q] = round(float(new_mark),1)  #add to dict rounded to 1 dp

In [13]:
print("Grade comparison after normalization:")
for student in sorted(grade_dict.keys()):
    print("%%%student:", student)
    print("original:",[(k, v) for k, v in sorted(grade_dict[student].items())])
    print("transformed:",[(k, v) for k, v in sorted(normalized_grade_dict[student].items())])

Grade comparison after normalization:
%%%student: group1
original: [('q1', 10.0), ('q2', 9.0), ('q3', 7.0)]
transformed: [('q1', 14.5), ('q2', 14.1), ('q3', 13.1)]
%%%student: group10
original: [('q1', 10.0), ('q2', 9.0), ('q3', 6.0)]
transformed: [('q1', 14.5), ('q2', 14.1), ('q3', 11.9)]
%%%student: group11
original: [('q1', 15.0), ('q2', 17.0), ('q3', 20.0)]
transformed: [('q1', 13.7), ('q2', 15.0), ('q3', 20.0)]
%%%student: group12
original: [('q1', 15.0), ('q2', 18.0), ('q3', 19.0)]
transformed: [('q1', 13.7), ('q2', 15.7), ('q3', 19.0)]
%%%student: group13
original: [('q1', 15.0), ('q2', 18.0), ('q3', 20.0)]
transformed: [('q1', 13.7), ('q2', 15.7), ('q3', 20.0)]
%%%student: group14
original: [('q1', 15.0), ('q2', 14.0), ('q3', 15.0)]
transformed: [('q1', 13.7), ('q2', 13.2), ('q3', 15.0)]
%%%student: group15
original: [('q1', 16.0), ('q2', 13.0), ('q3', 12.0)]
transformed: [('q1', 14.2), ('q2', 12.5), ('q3', 12.0)]
%%%student: group16
original: [('q1', 14.0), ('q2', 14.0), ('q3'

In [19]:
# replace old scores with new ones
for i in range(0, num_students):
    # iterate over the questions
    for q in questions:
        student = results_df['student_name'][i]
        results_df[q][i] = float(normalized_grade_dict[student][q])

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
results_df

,student_name,marker_name,q1,q2,q3
0,group1,john,14,14,13
1,group2,john,15,14,15
2,group3,john,13,14,16
3,group4,john,14,13,16
4,group5,john,14,12,19
5,group6,john,15,14,20
6,group7,john,16,13,21
7,group8,john,16,14,19
8,group9,john,14,9,16
9,group10,john,14,14,11


In [21]:
results_df.to_csv("results_normalized.csv") #save to file